<a href="https://colab.research.google.com/github/JimKing100/DS-Unit-2-Kaggle-Challenge/blob/master/Kaggle_Challenge_X1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Installs
%%capture
!pip install --upgrade category_encoders plotly

In [46]:
# Imports
import os, sys

os.chdir('/content')
!git init .
!git remote add origin https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge.git
!git pull origin master

!pip install -r requirements.txt

os.chdir('module1')

Reinitialized existing Git repository in /content/.git/
fatal: remote origin already exists.
From https://github.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge
 * branch            master     -> FETCH_HEAD
Already up to date.


In [0]:
# Imports
import pandas as pd
import numpy as np
import math

import sklearn
sklearn.__version__
from sklearn.model_selection import train_test_split

# Import the models
from sklearn.linear_model import LogisticRegressionCV
from sklearn.pipeline import make_pipeline

# Import encoder and scaler and imputer
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

# Import random forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [48]:
def wrangle(X):
    """Wrangles train, validate, and test sets in the same way"""
    X = X.copy()

    # Convert date_recorded to datetime
    X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)
    
    # Extract components from date_recorded, then drop the original column
    X['year_recorded'] = X['date_recorded'].dt.year
    X['month_recorded'] = X['date_recorded'].dt.month
    X['day_recorded'] = X['date_recorded'].dt.day
    X = X.drop(columns='date_recorded')
    
    # Engineer feature: how many years from construction_year to date_recorded
    X.loc[X['construction_year'] == 0, 'construction_year'] = np.nan
    X['years'] = X['year_recorded'] - X['construction_year']
    
    # Engineer new feature pump_age
    #X.loc[X['construction_year'] == 0, 'construction_year'] = np.nan
    #X['pump_age'] = 2013 - X['construction_year']
    
    #X['region_district'] = X['region_code'].astype(str) + X['district_code'].astype(str)
    
    # About 3% of the time, latitude has small values near zero,
    # outside Tanzania, so we'll treat these like null values
    X['latitude'] = X['latitude'].replace(-2e-08, np.nan)
    
    # When columns have zeros and shouldn't, they are like null values
    cols_with_zeros = ['construction_year', 'longitude', 'latitude', 'gps_height',
                       'population', 'amount_tsh']
    for col in cols_with_zeros:
        X[col] = X[col].replace(0, np.nan)
        
    X.loc[X['years'].isna(), 'years'] = X['years'].mean()
    #X.loc[X['pump_age'].isna(), 'pump_age'] = X['pump_age'].mean()
    
    # Impute mean for longitude and latitude based on region
    average_lat = X.groupby('region').latitude.mean().reset_index()
    average_long = X.groupby('region').longitude.mean().reset_index()

    shinyanga_lat = average_lat.loc[average_lat['region'] == 'Shinyanga', 'latitude']
    shinyanga_long = average_long.loc[average_lat['region'] == 'Shinyanga', 'longitude']

    X.loc[(X['region'] == 'Shinyanga') & (X['latitude'] > -1), ['latitude']] = shinyanga_lat[17]
    X.loc[(X['region'] == 'Shinyanga') & (X['longitude'].isna()), ['longitude']] = shinyanga_long[17]

    mwanza_lat = average_lat.loc[average_lat['region'] == 'Mwanza', 'latitude']
    mwanza_long = average_long.loc[average_lat['region'] == 'Mwanza', 'longitude']

    X.loc[(X['region'] == 'Mwanza') & (X['latitude'] > -1), ['latitude']] = mwanza_lat[13]
    X.loc[(X['region'] == 'Mwanza') & (X['longitude'].isna()) , ['longitude']] = mwanza_long[13]
    
    #X.loc[X['amount_tsh'].isna(), 'amount_tsh'] = 0
    
    # Clean installer
    X['installer'] = X['installer'].str.lower()
    X['installer'] = X['installer'].str[:3]
    X['installer'].value_counts(normalize=True)
    tops = X['installer'].value_counts()[:15].index
    X.loc[~X['installer'].isin(tops), 'installer'] = 'other'
    
    # Clean funder and bin
    #X['funder'] = X['funder'].str.lower()
    #X['funder'] = X['funder'].str[:3]
    #X['funder'].value_counts(normalize=True)
    #tops = X['funder'].value_counts()[:15].index
    #X.loc[~X['funder'].isin(tops), 'funder'] = 'other'
    
    # Bin lga
    #tops = X['lga'].value_counts()[:15].index
    #X.loc[~X['lga'].isin(tops), 'lga'] = 'Other'
    
    # Bin subvillage
    tops = X['subvillage'].value_counts()[:15].index
    X.loc[~X['subvillage'].isin(tops), 'subvillage'] = 'Other'
    
    # Impute mean for the feature based on latitude and longitude
    def latlong_conversion(feature, pop, long, lat):
    
      radius = 0.1
      radius_increment = 0.3
    
      if math.isnan(pop):
        pop_temp = 0
        while pop_temp <= 1 and radius <= 2:
          lat_from = lat - radius
          lat_to = lat + radius
          long_from = long - radius
          long_to = long + radius
        
          df = X[(X['latitude'] >= lat_from) & 
                 (X['latitude'] <= lat_to) &
                 (X['longitude'] >= long_from) &
                 (X['longitude'] <= long_to)]
        
          pop_temp = df[feature].mean()
          
          radius = radius + radius_increment
      else:
        pop_temp = pop
      
      if np.isnan(pop_temp):
        new_pop = X_train[feature].mean()
      else:
        new_pop = pop_temp
   
      return new_pop
    
    #X['population'] = X.apply(lambda x: latlong_conversion('population', x['population'], x['longitude'], x['latitude']), axis=1)
    
    # Impute mean for tsh based on mean of source_class/basin/waterpoint_type_group
    #def tsh_calc(tsh, source, base, waterpoint):
    #  if math.isnan(tsh):
    #    if (source, base, waterpoint) in tsh_dict:
    #      new_tsh = tsh_dict[source, base, waterpoint]
    #      return new_tsh
    #  else:
    #    return tsh
    #  return tsh
  
    #temp = X[~X['amount_tsh'].isna()].groupby(['source_class',
    #                                           'basin',
    #                                           'waterpoint_type_group'])['amount_tsh'].mean()
    #tsh_dict = dict(temp)
    #X['amount_tsh'] = X.apply(lambda x: tsh_calc(x['amount_tsh'], x['source_class'], x['basin'], x['waterpoint_type_group']), axis=1)
    
    # Drop unneeded columns
    unusable_variance = ['recorded_by', 'id', 'num_private', 'wpt_name']
    X = X.drop(columns=unusable_variance)
        
    return X

  # Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv('../data/tanzania/train_features.csv'), 
                 pd.read_csv('../data/tanzania/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv('../data/tanzania/test_features.csv')
sample_submission = pd.read_csv('../data/tanzania/sample_submission.csv')

# Split train into train & val. Make val the same size as test.
target = 'status_group'
train, val = train_test_split(train, test_size=len(test),  
                              stratify=train[target], random_state=42)

# Wrangle train, validate, and test sets in the same way
train = wrangle(train)
val = wrangle(val)
test = wrangle(test)

train.head(25)

,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,year_recorded,month_recorded,day_recorded,years
45793,NaN,NaN,NaN,other,33.075583,-9.385449,Lake Nyasa,Other,Mbeya,12,5,Ileje,Chitete,NaN,True,VWC,NaN,False,NaN,nira/tanira,nira/tanira,handpump,vwc,user-group,unknown,unknown,unknown,unknown,dry,dry,shallow well,shallow well,groundwater,hand pump,hand pump,non functional,2011,3,30,15.347859
26326,500.0,Dhv,285.0,dwe,36.228574,-8.207742,Rufiji,Other,Morogoro,5,3,Kilombero,Mbingu,1000.0,True,NaN,NaN,True,1984.0,swn 80,swn 80,handpump,vwc,user-group,pay monthly,monthly,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,non functional,2011,2,28,27.000000
53251,NaN,Finw,218.0,fin,39.673635,-10.835281,Ruvuma / Southern Coast,Other,Mtwara,9,4,Tandahimba,Naputa,260.0,True,Water Board,Borehole,True,1982.0,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,2013,1,27,31.000000
26791,500.0,Adb,1704.0,dwe,34.915589,-9.016965,Rufiji,Other,Iringa,11,4,Njombe,Igongolo,40.0,True,VWC,Ibiki gravity water scheme,False,2008.0,gravity,gravity,gravity,vwc,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,2011,2,27,3.000000
2162,200.0,Dwe,1232.0,dwe,30.332034,-4.308921,Lake Tanganyika,Other,Kigoma,16,2,Kasulu,Kitagata,500.0,True,Water authority,Nyachenda,True,2003.0,gravity,gravity,gravity,vwc,user-group,pay monthly,monthly,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,2013,1,18,10.000000
14686,NaN,NaN,NaN,other,32.926770,-9.317874,Lake Nyasa,Other,Mbeya,12,5,Ileje,Mbebe,NaN,True,VWC,NaN,False,NaN,nira/tanira,nira/tanira,handpump,vwc,user-group,unknown,unknown,soft,good,seasonal,seasonal,shallow well,shallow well,groundwater,hand pump,hand pump,functional,2011,3,27,15.347859
57063,NaN,Ded,NaN,dwe,30.521085,-2.856737,Lake Victoria,Other,Kagera,18,30,Ngara,Bukiriro,NaN,True,VWC,NaN,False,NaN,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional needs repair,2011,7,19,15.347859
58277,NaN,Government Of Tanzania,1164.0,dis,37.621413,-3.327062,Pangani,Other,Kilimanjaro,3,1,Rombo,Mamsera,1.0,True,Company,Hakwe water supply,True,1974.0,gravity,gravity,gravity,company,commercial,pay monthly,monthly,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe,non functional,2013,3,7,39.000000
18025,200.0,Po,426.0,other,37.135017,-6.685168,Wami / Ruvu,Shuleni,Morogoro,5,1,Kilosa,Chanzuru,120.0,True,VWC,NaN,True,1976.0,swn 80,swn 80,handpump,vwc,user-group,pay monthly,monthly,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional,2011,4,21,35.000000
55692,NaN,Tassaf,1313.0,other,30.873994,-3.372826,Lake Tanganyika,Other,Kigoma,16,1,Kibondo,Kasanda,350.0,True,VWC,NaN,False,2011.0,india mark ii,india mark ii,handpump,vwc,user-group,unknown,unknown,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,non functional,2013,2,8,2.000000


In [49]:
# Arrange data into X features matrix and y target vector
X_train = train.drop(columns=target)
y_train = train[target]
X_val = val.drop(columns=target)
y_val = val[target]
X_test = test

# Make pipeline!
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='mean'), 
    RandomForestClassifier(n_estimators=1400, 
                           random_state=42,
                           min_samples_split=5,
                           min_samples_leaf=1,
                           max_features='auto',
                           max_depth=30,
                           bootstrap=True,
                           n_jobs=-1,
                           verbose = 1)
)

# Fit on train, score on val
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_val)
print('Validation Accuracy', accuracy_score(y_val, y_pred))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    3.2s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:   31.5s
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:   56.0s
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1400 out of 1400 | elapsed:  1.6min finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    1.3s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    2.3s
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed:    3.6s


Validation Accuracy 0.8200306449366207


[Parallel(n_jobs=2)]: Done 1400 out of 1400 | elapsed:    4.0s finished


In [50]:
pd.set_option('display.max_rows', 200)
model = pipeline.named_steps['randomforestclassifier']
encoder = pipeline.named_steps['ordinalencoder']
encoded_columns = encoder.transform(X_train).columns 
importances = pd.Series(model.feature_importances_, encoded_columns)
importances.sort_values(ascending=False)

longitude                0.082370
latitude                 0.081614
quantity_group           0.079409
quantity                 0.074675
gps_height               0.045112
day_recorded             0.038517
ward                     0.036957
waterpoint_type          0.035683
years                    0.035116
construction_year        0.034271
funder                   0.031940
population               0.031719
waterpoint_type_group    0.031160
extraction_type_class    0.025873
scheme_name              0.023647
extraction_type          0.023565
lga                      0.022654
extraction_type_group    0.021319
amount_tsh               0.016803
payment                  0.016772
district_code            0.016510
installer                0.016434
payment_type             0.016423
source                   0.015426
month_recorded           0.015406
region_code              0.014739
region                   0.014322
basin                    0.013788
source_type              0.013492
management    

In [51]:
assert all(X_test.columns == X_train.columns)

y_pred = pipeline.predict(X_test)

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.6s
[Parallel(n_jobs=2)]: Done 446 tasks      | elapsed:    1.2s
[Parallel(n_jobs=2)]: Done 796 tasks      | elapsed:    2.2s
[Parallel(n_jobs=2)]: Done 1246 tasks      | elapsed:    3.5s
[Parallel(n_jobs=2)]: Done 1400 out of 1400 | elapsed:    3.9s finished


In [0]:
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('/content/submission-f1.csv', index=False)